<a href="https://colab.research.google.com/github/rubenguerra/NLP/blob/main/spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy

In [ ]:
!pip install -U spacy-lookups-data

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import tensorflow as tf

In [ ]:
physical_devices=tf.config.experimental.list_physical_devices('GPU')
print('Num GPUs disponible: ', len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs disponible:  1


In [ ]:
#Bajar la data
#Instalamos la biblioteca kaggle
!pip install kaggle

In [ ]:
#Descargamos el permiso de la cuenta kaggle (previamente bajado desde la url)
from google.colab import files
files.upload()

In [ ]:
#Antes de importar la dataset debemos usar este codigo
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#Este permiso cambia
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Importamos el dataset que queremoos usar un proyecto

!kaggle datasets download -d kazanova/sentiment140

401 - Unauthorized


In [ ]:
from zipfile import ZipFile
datos="sentiment140.zip"

with ZipFile(datos, 'r') as zip:
  zip.extractall()
  print('Listo')

In [ ]:
import pandas as pd
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import os

In [ ]:
ls

kaggle1.json  sample_data/      training.1600000.processed.noemoticon.csv
kaggle.json   sentiment140.zip


In [ ]:
os.rename(r'/content/training.1600000.processed.noemoticon.csv', r'/content/twitter16m.csv',)

In [ ]:
df=pd.read_csv('twitter16m.csv', encoding='latin1', header=None)

In [ ]:
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#SELECIONAMOS LAS COLUMNAS QUE NOS INTERESAN EN NUESTRO DF
df=df[[5,0]]

In [ ]:
#RENOMBRAMOS LAS COLUMNAS
df.columns=['twitts', 'Sent.']
df.head()

,twitts,Sent.
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [ ]:
df['Sent.'].value_counts()

4    800000
0    800000
Name: Sent., dtype: int64

In [ ]:
sent_map={0:'negativo', 4:'positivo'} #ETIQUETAMOS LOS VALORES

CONTEO DE PALABRAS

In [ ]:
df['cant_palabra']=df['twitts'].apply(lambda x: len(str(x).split()))

In [ ]:
df['cant_car']=df['twitts'].apply(lambda x: len(x))

In [ ]:
def get_avg_word_len(x):
  words=x.split()
  word_len=0
  for word in words:
    word_len=word_len + len(word)
  return word_len/len(words)

In [ ]:
df['prom_log_pal']=df['twitts'].apply(lambda x: get_avg_word_len(x))

In [ ]:
df.head()

,twitts,Sent.,cant_palabra,cant_car,prom_log_pal
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19,115,5.052632
1,is upset that he can't update his Facebook by ...,0,21,111,4.285714
2,@Kenichan I dived many times for the ball. Man...,0,18,89,3.944444
3,my whole body feels itchy and like its on fire,0,10,47,3.700000
4,"@nationwideclass no, it's not behaving at all....",0,21,111,4.285714


CONTEO STOP WORDS

In [ ]:
df['cant_stopWords']=df['twitts'].apply(lambda x: len([t for t in x.split() if t in STOP_WORDS]))

In [ ]:
df.head()

,twitts,Sent.,cant_palabra,cant_car,prom_log_pal,cant_stopWords
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19,115,5.052632,4
1,is upset that he can't update his Facebook by ...,0,21,111,4.285714,9
2,@Kenichan I dived many times for the ball. Man...,0,18,89,3.944444,7
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5
4,"@nationwideclass no, it's not behaving at all....",0,21,111,4.285714,10


CONTEO HASHTAG Y MENCIONES

In [ ]:
df['hashtags']=df['twitts'].apply(lambda x: len([t for t in x.split() if t.startswith('#')]))
df['menciones']=df['twitts'].apply(lambda x: len([t for t in x.split() if t.startswith('@')]))
df.head()

,twitts,Sent.,cant_palabra,cant_car,prom_log_pal,cant_stopWords,hashtags,menciones
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19,115,5.052632,4,0,1
1,is upset that he can't update his Facebook by ...,0,21,111,4.285714,9,0,0
2,@Kenichan I dived many times for the ball. Man...,0,18,89,3.944444,7,0,1
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0
4,"@nationwideclass no, it's not behaving at all....",0,21,111,4.285714,10,0,1


DICE SI HAY DIGITOS EN LOS TWITTS

In [ ]:
df['cant_digit']=df['twitts'].apply(lambda x: len([t for t in x.split() if t.isdigit()]))

In [ ]:
df.head()

,twitts,Sent.,cant_palabra,cant_car,prom_log_pal,cant_stopWords,hashtags,menciones,cant_digit
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19,115,5.052632,4,0,1,0
1,is upset that he can't update his Facebook by ...,0,21,111,4.285714,9,0,0,0
2,@Kenichan I dived many times for the ball. Man...,0,18,89,3.944444,7,0,1,0
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0,0
4,"@nationwideclass no, it's not behaving at all....",0,21,111,4.285714,10,0,1,0


CONTEO DE PALABRAS EN MAYUSCULAS

In [ ]:
df['cant_may']=df['twitts'].apply(lambda x: len([t for t in x.split() if t.isupper() and len(x) > 3  ]))

In [ ]:
df.head()

,twitts,Sent.,cant_palabra,cant_car,prom_log_pal,cant_stopWords,hashtags,menciones,cant_digit,cant_may
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19,115,5.052632,4,0,1,0,1
1,is upset that he can't update his Facebook by ...,0,21,111,4.285714,9,0,0,0,0
2,@Kenichan I dived many times for the ball. Man...,0,18,89,3.944444,7,0,1,0,1
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0,0,0
4,"@nationwideclass no, it's not behaving at all....",0,21,111,4.285714,10,0,1,0,1


PREPROCESAMIENTO Y LIMPIEZA

Conversion a minusculas

In [ ]:
df['twitts']=df['twitts'].apply(lambda x: x.lower())

In [ ]:
df.head()

,twitts,Sent.,cant_palabra,cant_car,prom_log_pal,cant_stopWords,hashtags,menciones,cant_digit,cant_may
0,"@switchfoot http://twitpic.com/2y1zl - awww, t...",0,19,115,5.052632,4,0,1,0,1
1,is upset that he can't update his facebook by ...,0,21,111,4.285714,9,0,0,0,0
2,@kenichan i dived many times for the ball. man...,0,18,89,3.944444,7,0,1,0,1
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0,0,0
4,"@nationwideclass no, it's not behaving at all....",0,21,111,4.285714,10,0,1,0,1


CONTRACCIONES USADAS Y SU TRANSFORMACION

In [ ]:
contracciones={
    "ain't":"am not",
    "aren't":"are not",
    "can't": "cannot",
    "can't've":"cannot have",
    "'cause": "because",
    "could've":"could have",
    "couldn't":"could not",
    "couldn't've":"could not have",
    "didn't":"did not",
    "doesn't":"does not",
    "don't":"do not",
    "hadn't":"had not",
    "hadn't've":"had not have",
    "hasn't": "has not",
    "haven't":"have not",
    "he'd":"he would",
    "he'd've":"he would have",
    "he'll":"he will",
    "he'll've":"he will have",
    "he's":"he is",
    "how'd":"how did",
    "how'd'y":"how do you",
    "how'll":"how will",
    "how's":"how does",
    "i'd":"i would",
    "i'd've":"i would have",
    "i'll":"i will",
    "i'll've":"i will have",
    "i'm":"i am",
    "i've":"i have",
    "isn't":"is not",
    "it'd":"it would",
    "it'd've":"it would have",
    "it'll":"it will",
    "it'll've":"it will have",
    "it's":"it is",
    "let's":"let us",
    "ma'am":"madam",
    "mayn't":"may not",
    "might've": "might have",
    "mightn't":"might not",
    "mightn't've":"might not have",
    "must've":"must have",
    "mustn't":"must not",
    "mustn't've":"must not have",
    "needn't":"need not",
    "needn't've":"need not have",
    "o'clock":"of the clock",
    "oughtn't":"ought not",
    "oughtn't've":"ought not have",
    "shan't":"shall not",
    "sha'n't":"shall not",
    "shan't've":"shall not have",
    "she'd":"she would",
    "she'd've":"she would have",
    "she'll":"she will",
    "she'll've":"she will have",
    "she's":"she is",
    "should've":"should have",
    "shouldn't":"should not",
    "shouldn't've":"should not have",
    "so've":"so have",
    "so's":"so is",
    "that'd":"that would",
    "that'd've":"that would have",
    "that's":"that is",
    "there'd":"there would",
    "there'd've":"there would have",
    "there's":"there is",
    "they'd":"they would",
    "they'd've":"they would have",
    "they'll":"they will",
    "they'll've":"they will have",
    "they're":"they are",
    "they've":"they have",
    "to've":"to have",
    "wasn't":"was not",
    " u ":"you",
    " ur ":"your",
    " n ":"and"

}

In [ ]:
def cont_to_exp(x):
  if type(x) is str:
    for key in contracciones:
      value=contracciones[key]
      x=x.replace(key, value)
    return x
  else:
    return x

In [ ]:
x="hi, i'd be happy"

In [ ]:
cont_to_exp(x)

'hi, i would be happy'

In [ ]:
%%time
df['twitts']=df['twitts'].apply(lambda x: cont_to_exp(x))

CPU times: user 34.9 s, sys: 48.1 ms, total: 34.9 s
Wall time: 35 s


In [ ]:
df.head()

,twitts,Sent.,cant_palabra,cant_car,prom_log_pal,cant_stopWords,hashtags,menciones,cant_digit,cant_may
0,"@switchfoot http://twitpic.com/2y1zl - awww, t...",0,19,115,5.052632,4,0,1,0,1
1,is upset that he cannot update his facebook by...,0,21,111,4.285714,9,0,0,0,0
2,@kenichan i dived many times for the ball. man...,0,18,89,3.944444,7,0,1,0,1
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0,0,0
4,"@nationwideclass no, it is not behaving at all...",0,21,111,4.285714,10,0,1,0,1


CONTEO Y ELIMINACION DE EMAILS

In [ ]:
import re

In [ ]:
df['emails']=df['twitts'].apply(lambda x: re.findall(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)',x))

In [ ]:
df['cant_emails']=df['emails'].apply(lambda x: len(x))

In [ ]:
df[df['cant_emails'] > 0]

,twitts,Sent.,cant_palabra,cant_car,prom_log_pal,cant_stopWords,hashtags,menciones,cant_digit,cant_may,emails,cant_emails
4054,i want a new laptop. hp tx2000 is the bomb. :...,0,20,103,4.150000,6,0,0,0,4,[gabbehhramos@yahoo.com],1
7917,who stole elledell@gmail.com?,0,3,31,9.000000,1,0,0,0,0,[elledell@gmail.com],1
8496,@alexistehpom really? did you send out all th...,0,20,130,5.500000,11,0,1,0,0,[missataari@gmail.com],1
10290,@laureystack awh...that is kinda sad lol add ...,0,8,76,8.500000,0,0,1,0,0,[hello.kitty.65@hotmail.com],1
16413,"@jilliancyork got 2 bottom of it, human error...",0,21,137,5.428571,7,0,1,1,0,[press@linkedin.com],1
...,...,...,...,...,...,...,...,...,...,...,...,...
1588338,@boudoirsextoys do not forget to shoot me an e...,4,21,127,5.047619,10,0,1,0,1,[hello@tastelikekisses.com],1
1589132,"hi there , anybody got a job for me oil &amp;...",4,15,98,5.533333,4,0,0,0,0,[aadvanspijk@yahoo.co.uk],1
1590489,"@clericaaron yes, lbruton@tulsalanparty.com is...",4,14,95,5.785714,6,0,1,0,1,[lbruton@tulsalanparty.com],1
1591675,@miss_ellen good morning!!!! congrats! se...,4,9,77,7.111111,2,0,1,0,0,[jill@q985fm.com],1


ELIMINACION DEL EMAILS EN EL TWITT

In [ ]:
re.sub(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)','',x)

"hi, i'd be happy"

In [ ]:
df['twitts']=df['twitts'].apply(lambda x: re.sub(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)','',x))

In [ ]:
df[df['cant_emails'] > 0].head()

,twitts,Sent.,cant_palabra,cant_car,prom_log_pal,cant_stopWords,hashtags,menciones,cant_digit,cant_may,emails,cant_emails
4054,i want a new laptop. hp tx2000 is the bomb. :...,0,20,103,4.150000,6,0,0,0,4,[gabbehhramos@yahoo.com],1
7917,who stole ?,0,3,31,9.000000,1,0,0,0,0,[elledell@gmail.com],1
8496,@alexistehpom really? did you send out all th...,0,20,130,5.500000,11,0,1,0,0,[missataari@gmail.com],1
10290,@laureystack awh...that is kinda sad lol add ...,0,8,76,8.500000,0,0,1,0,0,[hello.kitty.65@hotmail.com],1
16413,"@jilliancyork got 2 bottom of it, human error...",0,21,137,5.428571,7,0,1,1,0,[press@linkedin.com],1


CONTEO DE URLS Y ELIMINACION DEL TWITT

In [ ]:
x='hi, to watch more visit https://youtube.com/kgptalkie'

In [ ]:
re.findall(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])', x)

[('https', 'youtube.com', '/kgptalkie')]

In [ ]:
df['urls_flag']=df['twitts'].apply(lambda x: len(re.findall(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])', x)))

In [ ]:
re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])','',x)

'hi, to watch more visit '

In [ ]:
df['twitts']=df['twitts'].apply(lambda x: re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])','',x))

In [ ]:
df.head()

,twitts,Sent.,cant_palabra,cant_car,prom_log_pal,cant_stopWords,hashtags,menciones,cant_digit,cant_may,emails,cant_emails,urls_flag
0,"@switchfoot - awww, that is a bummer. you sh...",0,19,115,5.052632,4,0,1,0,1,[],0,1
1,is upset that he cannot update his facebook by...,0,21,111,4.285714,9,0,0,0,0,[],0,0
2,@kenichan i dived many times for the ball. man...,0,18,89,3.944444,7,0,1,0,1,[],0,0
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0,0,0,[],0,0
4,"@nationwideclass no, it is not behaving at all...",0,21,111,4.285714,10,0,1,0,1,[],0,0


In [ ]:
df.loc[0]['twitts']

'@switchfoot  - awww, that is a bummer.  you shoulda got david carr of third day to do it. ;d'

ELIMINACION DE RT

In [ ]:
df['twitts']=df['twitts'].apply(lambda x: re.sub('RT', '',x))

ELIMINACION DE CARACTERES ESPECIALES O SIGNOS DE PUNTUACION

In [ ]:
df['twitts']=df['twitts'].apply(lambda x: re.sub('[^A-Z a-z 0-9-]+','',x))

In [ ]:
df.head()

,twitts,Sent.,cant_palabra,cant_car,prom_log_pal,cant_stopWords,hashtags,menciones,cant_digit,cant_may,emails,cant_emails,urls_flag
0,switchfoot - awww that is a bummer you shoul...,0,19,115,5.052632,4,0,1,0,1,[],0,1
1,is upset that he cannot update his facebook by...,0,21,111,4.285714,9,0,0,0,0,[],0,0
2,kenichan i dived many times for the ball manag...,0,18,89,3.944444,7,0,1,0,1,[],0,0
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0,0,0,[],0,0
4,nationwideclass no it is not behaving at all i...,0,21,111,4.285714,10,0,1,0,1,[],0,0


REMOVIENDO ESPACIOS MULTIPLES

In [ ]:
x='thanks      for watching and     please like this video'

In [ ]:
" ".join(x.split())

'thanks for watching and please like this video'

In [ ]:
df['twitts']=df['twitts'].apply(lambda x: " ".join(x.split()))

In [ ]:
df.head(2)

,twitts,Sent.,cant_palabra,cant_car,prom_log_pal,cant_stopWords,hashtags,menciones,cant_digit,cant_may,emails,cant_emails,urls_flag
0,switchfoot - awww that is a bummer you shoulda...,0,19,115,5.052632,4,0,1,0,1,[],0,1
1,is upset that he cannot update his facebook by...,0,21,111,4.285714,9,0,0,0,0,[],0,0


BORRADO DE TAGS HTML

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
%%time
df['twitts']=df['twitts'].apply(lambda x: BeautifulSoup(x,'lxml').get_text())

CPU times: user 5min 59s, sys: 25.8 s, total: 6min 25s
Wall time: 6min 25s


REMOVIENDO CARACTERES DE ACENTUACION

In [ ]:
x='Áccënted töxt'

In [ ]:
import unicodedata

In [ ]:
def remove_accented_chars(x):
  x=unicodedata.normalize('NFKD',x).encode('ascii', 'ignore').decode('utf-8','ignore')
  return x

In [ ]:
remove_accented_chars(x)

'Accented toxt'

SPACY Y NLP

BORRAR STOP WORDS

In [ ]:
x='this is stop words removal code'

In [ ]:
" ".join([t for t in x.split() if t not in STOP_WORDS])

'stop words removal code'

In [ ]:
df['twitts']=df['twitts'].apply(lambda x: " ".join([t for t in x.split() if t not in STOP_WORDS]))

In [ ]:
df.head()

,twitts,Sent.,cant_palabra,cant_car,prom_log_pal,cant_stopWords,hashtags,menciones,cant_digit,cant_may,emails,cant_emails,urls_flag
0,switchfoot - awww bummer shoulda got david car...,0,19,115,5.052632,4,0,1,0,1,[],0,1
1,upset update facebook texting cry result schoo...,0,21,111,4.285714,9,0,0,0,0,[],0,0
2,kenichan dived times ball managed save 50 rest...,0,18,89,3.944444,7,0,1,0,1,[],0,0
3,body feels itchy like fire,0,10,47,3.700000,5,0,0,0,0,[],0,0
4,nationwideclass behaving mad,0,21,111,4.285714,10,0,1,0,1,[],0,0


CONVIRTIENDO PALABRAS EN SU FORMA BASE O RAIZ

In [ ]:
nlp=spacy.load('en_core_web_sm')

In [ ]:
x='kenichan dived times ball managed save 50 rest'

dive= dived, time = times, manage= managed

In [ ]:
#x='i you he she they is am are'

In [ ]:
def make_to_base(x):
  x_list=[]
  doc=nlp(x)

  for token in doc:
    lemma = str(token.lemma_)
    if lemma == '-PRON-' or lemma=='be':
      lemma=token.text
    x_list.append(lemma)
  print(" ".join(x_list))


In [ ]:
make_to_base(x)

kenichan dive times ball manage save 50 rest


ELIMINACION DE PALABRAS COMUNES

In [ ]:
' '.join(df.head()['twitts'])

'switchfoot - awww bummer shoulda got david carr day d upset update facebook texting cry result school today blah kenichan dived times ball managed save 50 rest bounds body feels itchy like fire nationwideclass behaving mad'

In [ ]:
text=' '.join(df['twitts']) 

In [ ]:
text=text.split()

In [ ]:
freq_comm=pd.Series(text).value_counts()

In [ ]:
f20=freq_comm[:20]

In [ ]:
f20

good      89227
day       81969
like      76851
-         69467
today     64348
going     63815
work      62611
love      61913
got       59997
time      55863
lol       54651
im        50004
know      49505
new       41803
want      41514
night     40946
think     40326
amp       40268
thanks    39254
home      39051
dtype: int64

In [ ]:
df['twitts']=df['twitts'].apply(lambda)